In [66]:
#!/usr/bin/env python
# coding: utf-8
import sys
import os.path
os.environ['TZ'] = 'Asia/Calcutta'
import pandas as pd
import numpy as np
from google.cloud import bigquery
import os
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import warnings
import json
warnings.filterwarnings("ignore")
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/foram/Desktop/Foram/MyGlamm - Data Storage-3800e204ff33.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = base_dir + "/core/.mg_in_creds.json"
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/centos/data_transform/core/.mg_in_creds.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/Mouna Reddy/OneDrive/Documents/mouna_reddy.json"
client=bigquery.Client()

In [67]:
# query1 = '''select * from 
#             `myglamm-india.bq_audit_logs.pra_product_details_table_v2` limit 10; '''
# df1 = client.query(query1).to_dataframe()
# df1

In [68]:
query1 = ('''
    with raw_data as (select 
        * 
    from (
        select 
            sku, 
            sku_name,
            parent_category, 
            category,
            sub_category, 
            product, 
            base_price, 
            brand_name,
            concat(
                case 
                    when sku_name is null then '' 
                    else sku_name 
                end, 
                ' ', 
                
                case 
                    when category is null then '' 
                    else category 
                end, 
                ' ',
                case 
                    when sub_category is null then '' 
                    else sub_category 
                end, 
                ' ',
                case 
                    when finish is null then '' 
                    else finish 
                end , 
                ' ',
                case 
                    when formulation is null then '' 
                    else formulation 
                end, 
                ' ',
                case 
                    when color is null then '' 
                    else color 
                end , 
                ' ',
                case 
                    when brand_name is null then '' 
                    else brand_name 
                end 
            ) as Description 
        from 
            `myglamm-india.bq_audit_logs.pra_product_details_table_v2` 
        where 
            stock >= 0 
            and product_status='Active' 
            and type!='combo' 
            and base_price != 0
    ) 
    where 
        Description is not null),

 
vcode as (SELECT oa.* , ct.vendor_code FROM raw_data oa
left join (select * from `myglamm-india.master_tables.master_product_table_v2` where product_status='Active' 
        and type!='combo' ) as ct
on oa.sku = ct.sku)

select * from vcode where vendor_code = 'mgp';

''')


In [69]:
query_non_oos = ('''
    select 
        sku,
        sku_name 
    from 
        `myglamm-india.bq_audit_logs.pra_product_details_table_v2` 
    where 
        stock_preorder in (
            'oos_pre_order', 
            'instock_pre_order',
            'instock_non_pre_order'
        ) 
        and product_status='Active' 
        and type!='combo' 
        and base_price!=0
''')

In [70]:
df1 = client.query(query1).to_dataframe()
df1

,sku,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code
0,8076149815,Manish Malhotra Powder Matte Lipstick,Makeup,Lip Makeup,Matte Lipstick,Manish Malhotra Powder Matte Lipstick,1.0,Manish Malhotra,Manish Malhotra Powder Matte Lipstick Lip Make...,mgp
1,9790153431,Manish Malhotra Gold Flake Lipgloss,Makeup,Lip Makeup,Lip Gloss,Manish Malhotra Gold Flake Lipgloss,1.0,Manish Malhotra,Manish Malhotra Gold Flake Lipgloss Lip Makeup...,mgp
2,80761498143,Manish Malhotra Strobing Cream,Makeup,Face Makeup,Highlighter,Manish Malhotra Strobing Cream,1.0,Manish Malhotra,Manish Malhotra Strobing Cream Face Makeup Hig...,mgp
3,STPL1707001,MyGlamm Vanity Box,Makeup,Makeup Brushes And Accessories,Accessories,MyGlamm Vanity Box,2000.0,MyGlamm,MyGlamm Vanity Box Makeup Brushes And Accessor...,mgp
4,8906095887965,MyGlamm Avacado & Aloe Eye Cream - 30 Gm,Skincare,Eye Care,Eye Cream,Avacado & Aloe Eye Cream,499.0,MyGlamm,MyGlamm Avacado & Aloe Eye Cream - 30 Gm Eye C...,mgp
...,...,...,...,...,...,...,...,...,...,...
870,sgl8906095880416,TWO OF YOUR KIND -Ichor,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Ichor Nail Polish Sets Nail ...,mgp
871,sgl8906095880423,TWO OF YOUR KIND -True Blue,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -True Blue Nail Polish Sets N...,mgp
872,sgl8906095880430,TWO OF YOUR KIND -Oh My Gold,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Oh My Gold Nail Polish Sets ...,mgp
873,sgl8906095880447,TWO OF YOUR KIND -Saffron,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Saffron Nail Polish Sets Nai...,mgp


In [71]:
df1.Description[100]

'MyGlamm LIT Satin Matte Lipstick - Modern Family Lip Makeup Matte Lipstick | Lipstick Satin Creamy Red LIT'

In [72]:
df_normal_sku = client.query(query_non_oos).to_dataframe()
df_normal_sku

,sku,sku_name
0,600310067627,100% Natural Mosquito Repellent Patches
1,600310067634,100% Natural Vapour Patch
2,600310067641,Natural Foaming Handwash 200ml
3,600310067665,Organic Coconut Oil 100ml
4,600310067672,Nourishing Baby massage Oil 100ml
...,...,...
641,STPL-1835-001,Manish Malhotra Face Mask
642,STPL-1855-001,Myglamm Superfoods Pouch
643,STPL-1856-001,Lip Balm Pouch
644,89060958454657,Total Makeover FF Cream - Cocoa


In [73]:
query2= """
    select distinct 
        sku, 
        category_name,
        product_name
    from 
        `myglamm-india.master_tables.master_product_table_v2` 
    where 
        product_status='Active' 
        and type!='combo' 
        and vendor_code='mgp'
"""

df_product_category = client.query(query2).to_dataframe()

In [74]:
df_product_category

,sku,category_name,product_name
0,8906095881932,Manish Malhotra Hi-Shine Lipgloss,Manish Malhotra Hi-Shine Lipgloss - Retro Romance
1,8906095881925,Manish Malhotra Hi-Shine Lipgloss,Manish Malhotra Hi-Shine Lipgloss - Modern Muse
2,8906095881727,Manish Malhotra Nail Lacquer,Manish Malhotra Nail Lacquer - Opulent Ocean
3,8906095882083,LIT Lip & Eye Sparkles,MyGlamm LIT Lip & Eye Sparkles - Regina
4,8906095882137,LIT Satin Matte Lipstick,MyGlamm LIT Satin Matte Lipstick - The Affair
...,...,...,...
874,8906095887309,Wanderlust Chrome Matte Nail Polish,Wanderlust Chrome Matte Nail Polish - Cancun
875,8906095885244,Manish Malhotra Hi-Shine Lipstick,Manish Malhotra Hi-Shine Lipstick - Cinnamon C...
876,8906095888450,LIT Liquid Matte Lipstick,MyGlamm LIT Liquid Matte Lipstick - Respek
877,8906095887873,Two of Your Kind,MyGlamm Two of your kind Nail Enamel - Wine Ta...


In [75]:
df1.set_index('sku', inplace = True)
df1

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code
sku,,,,,,,,,
8076149815,Manish Malhotra Powder Matte Lipstick,Makeup,Lip Makeup,Matte Lipstick,Manish Malhotra Powder Matte Lipstick,1.0,Manish Malhotra,Manish Malhotra Powder Matte Lipstick Lip Make...,mgp
9790153431,Manish Malhotra Gold Flake Lipgloss,Makeup,Lip Makeup,Lip Gloss,Manish Malhotra Gold Flake Lipgloss,1.0,Manish Malhotra,Manish Malhotra Gold Flake Lipgloss Lip Makeup...,mgp
80761498143,Manish Malhotra Strobing Cream,Makeup,Face Makeup,Highlighter,Manish Malhotra Strobing Cream,1.0,Manish Malhotra,Manish Malhotra Strobing Cream Face Makeup Hig...,mgp
STPL1707001,MyGlamm Vanity Box,Makeup,Makeup Brushes And Accessories,Accessories,MyGlamm Vanity Box,2000.0,MyGlamm,MyGlamm Vanity Box Makeup Brushes And Accessor...,mgp
8906095887965,MyGlamm Avacado & Aloe Eye Cream - 30 Gm,Skincare,Eye Care,Eye Cream,Avacado & Aloe Eye Cream,499.0,MyGlamm,MyGlamm Avacado & Aloe Eye Cream - 30 Gm Eye C...,mgp
...,...,...,...,...,...,...,...,...,...
sgl8906095880416,TWO OF YOUR KIND -Ichor,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Ichor Nail Polish Sets Nail ...,mgp
sgl8906095880423,TWO OF YOUR KIND -True Blue,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -True Blue Nail Polish Sets N...,mgp
sgl8906095880430,TWO OF YOUR KIND -Oh My Gold,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Oh My Gold Nail Polish Sets ...,mgp


In [76]:
tokenizer = RegexpTokenizer("[\w']+") 
tokenizer

RegexpTokenizer(pattern="[\\w']+", gaps=False, discard_empty=True, flags=re.UNICODE|re.MULTILINE|re.DOTALL)

In [77]:
df1["lower"] = df1["Description"].apply(lambda x : x.lower())
df1

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower
sku,,,,,,,,,,
8076149815,Manish Malhotra Powder Matte Lipstick,Makeup,Lip Makeup,Matte Lipstick,Manish Malhotra Powder Matte Lipstick,1.0,Manish Malhotra,Manish Malhotra Powder Matte Lipstick Lip Make...,mgp,manish malhotra powder matte lipstick lip make...
9790153431,Manish Malhotra Gold Flake Lipgloss,Makeup,Lip Makeup,Lip Gloss,Manish Malhotra Gold Flake Lipgloss,1.0,Manish Malhotra,Manish Malhotra Gold Flake Lipgloss Lip Makeup...,mgp,manish malhotra gold flake lipgloss lip makeup...
80761498143,Manish Malhotra Strobing Cream,Makeup,Face Makeup,Highlighter,Manish Malhotra Strobing Cream,1.0,Manish Malhotra,Manish Malhotra Strobing Cream Face Makeup Hig...,mgp,manish malhotra strobing cream face makeup hig...
STPL1707001,MyGlamm Vanity Box,Makeup,Makeup Brushes And Accessories,Accessories,MyGlamm Vanity Box,2000.0,MyGlamm,MyGlamm Vanity Box Makeup Brushes And Accessor...,mgp,myglamm vanity box makeup brushes and accessor...
8906095887965,MyGlamm Avacado & Aloe Eye Cream - 30 Gm,Skincare,Eye Care,Eye Cream,Avacado & Aloe Eye Cream,499.0,MyGlamm,MyGlamm Avacado & Aloe Eye Cream - 30 Gm Eye C...,mgp,myglamm avacado & aloe eye cream - 30 gm eye c...
...,...,...,...,...,...,...,...,...,...,...
sgl8906095880416,TWO OF YOUR KIND -Ichor,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Ichor Nail Polish Sets Nail ...,mgp,two of your kind -ichor nail polish sets nail ...
sgl8906095880423,TWO OF YOUR KIND -True Blue,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -True Blue Nail Polish Sets N...,mgp,two of your kind -true blue nail polish sets n...
sgl8906095880430,TWO OF YOUR KIND -Oh My Gold,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Oh My Gold Nail Polish Sets ...,mgp,two of your kind -oh my gold nail polish sets ...


In [78]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df=0, stop_words='english',token_pattern=r"(?u)\S\S+")
tf

TfidfVectorizer(min_df=0, stop_words='english', token_pattern='(?u)\\S\\S+')

In [79]:
tfidf_matrix = tf.fit_transform(df1['lower'].tolist())
tfidf_matrix

<875x1083 sparse matrix of type '<class 'numpy.float64'>'
	with 8480 stored elements in Compressed Sparse Row format>

In [80]:
feature_names = tf.get_feature_names()
corpus_index = [n for n in df1['lower'].tolist()]
import pandas as pd
dftfidf_matrix = pd.DataFrame(tfidf_matrix.T.todense(), index=feature_names, columns=corpus_index)
dftfidf_matrix

,manish malhotra powder matte lipstick lip makeup matte lipstick manish malhotra,manish malhotra gold flake lipgloss lip makeup lip gloss manish malhotra,manish malhotra strobing cream face makeup highlighter manish malhotra,myglamm vanity box makeup brushes and accessories accessories myglamm,myglamm avacado & aloe eye cream - 30 gm eye care eye cream myglamm,folding carry bag makeup brushes and accessories makeup bags myglamm,coin pouch makeup brushes and accessories pouch myglamm,trendy make up pouch makeup brushes and accessories pouch myglamm,double decker trousseau box makeup brushes and accessories pouch myglamm,perfect curves matte lip crayon - biscotti old lip makeup matte lipstick | lipstick matte creamy myglamm,...,two of your kind -mauve it nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -melon mania nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -alarmed nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -tourmaline nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -nude is nice nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -ichor nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -true blue nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -oh my gold nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -saffron nail polish sets nail polish liquid + powder double ended myglamm,two of your kind -peaches and cream nail polish sets nail polish liquid + powder double ended myglamm
-alarmed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.537771,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-aquadisiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-bubble,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-candy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-cherry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"you,",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
youthfull,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yummy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zaddy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
normal_sku=df_normal_sku['sku'].values
normal_sku

array(['600310067627', '600310067634', '600310067641', '600310067665',
       '600310067672', '600310067689', '600310067733', '616967453669',
       '8906080036682', '8908003416755', '8906095880461', '8906095880515',
       '8906095880607', '8906095880614', '8906095880621', '8906095880942',
       '8906095881130', '8906095881147', '8906095881154', '8906095881161',
       '8906095881178', '8906095881185', '8906095881192', '8906095881208',
       '8906095881215', '8906095881222', '8906095881239', '8906095881246',
       '8906095881253', '8906095881260', '8906095881277', '8906095881284',
       '8906095881291', '8906095881307', '8906095881314', '8906095881338',
       '8906095881345', '8906095881352', '8906095881369', '8906095881376',
       '8906095881383', '8906095881390', '8906095881406', '8906095881413',
       '8906095881420', '8906095881437', '8906095881451', '8906095881468',
       '8906095881475', '8906095881482', '8906095881499', '8906095881505',
       '8906095881512', '89060958

In [82]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_similarities

array([[1.        , 0.44137232, 0.43031105, ..., 0.08006062, 0.08755335,
        0.08249048],
       [0.44137232, 1.        , 0.31072345, ..., 0.17312333, 0.        ,
        0.        ],
       [0.43031105, 0.31072345, 1.        , ..., 0.        , 0.        ,
        0.12612213],
       ...,
       [0.08006062, 0.17312333, 0.        , ..., 1.        , 0.64997275,
        0.61238738],
       [0.08755335, 0.        , 0.        , ..., 0.64997275, 1.        ,
        0.66969956],
       [0.08249048, 0.        , 0.12612213, ..., 0.61238738, 0.66969956,
        1.        ]])

In [83]:
indices = pd.Series(df1.index) #indices[indices['sku'] == 'ORBOT13']
indices
indices[indices == '8076149815'].index[0]

0

In [84]:
indices[indices == '8076149815']

0    8076149815
Name: sku, dtype: object

In [85]:
# recommeded_order_sku = recommendations(indices[109],'category')
# recommeded_order_sku

In [86]:
# remove_same_category_products(recommeded_order_sku)

In [87]:
indices[25]
idx = indices[indices == '8076149815'].index[0]

In [88]:
score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
score_series

0      1.000000
334    0.761270
335    0.743689
330    0.733126
146    0.721161
         ...   
741    0.000000
742    0.000000
743    0.000000
744    0.000000
437    0.000000
Length: 875, dtype: float64

In [89]:
all_indexes = list(score_series.index)
all_indexes

[0,
 334,
 335,
 330,
 146,
 140,
 113,
 110,
 147,
 333,
 142,
 145,
 225,
 336,
 106,
 117,
 503,
 108,
 332,
 226,
 143,
 118,
 111,
 227,
 112,
 114,
 148,
 331,
 109,
 116,
 115,
 141,
 149,
 337,
 105,
 144,
 119,
 139,
 107,
 565,
 200,
 201,
 199,
 198,
 764,
 566,
 374,
 501,
 765,
 564,
 405,
 500,
 763,
 498,
 767,
 407,
 409,
 410,
 766,
 411,
 412,
 403,
 499,
 339,
 502,
 408,
 402,
 406,
 404,
 846,
 1,
 2,
 414,
 305,
 360,
 416,
 310,
 309,
 294,
 308,
 307,
 311,
 306,
 359,
 312,
 298,
 322,
 282,
 321,
 287,
 85,
 413,
 197,
 196,
 84,
 415,
 423,
 127,
 838,
 232,
 231,
 228,
 837,
 839,
 124,
 128,
 123,
 122,
 470,
 840,
 841,
 644,
 129,
 125,
 338,
 344,
 238,
 236,
 202,
 769,
 121,
 190,
 66,
 169,
 158,
 159,
 160,
 164,
 343,
 342,
 165,
 166,
 167,
 168,
 162,
 156,
 188,
 189,
 191,
 192,
 193,
 194,
 527,
 528,
 529,
 157,
 120,
 130,
 126,
 830,
 363,
 302,
 358,
 214,
 220,
 233,
 768,
 280,
 290,
 286,
 467,
 464,
 460,
 465,
 466,
 218,
 362,
 354,
 

In [90]:
global recommended_prods
recommended_prods = []
recommended_indices = []
recommeded_scores = []
recommeded_order_sku = []

In [91]:
remove_filter = 'category'
for i in all_indexes:
    if df1['product'][idx] != df1['product'][i]:
        if df1[remove_filter][idx] == df1[remove_filter][i]:
            recommended_prods.append((df1.index)[i])
            recommended_indices.append(i)
            recommeded_scores.append(score_series[i])
            #j = j + 1

In [92]:
# df1[remove_filter][idx] == df1[remove_filter][1589]

In [93]:
df1['category'][idx]

'Lip Makeup'

In [94]:
# df1[remove_filter][1589]

In [95]:
recommeded_scores

[0.7612698145945416,
 0.7436890154953251,
 0.7331260665495274,
 0.7211605240375691,
 0.7149865737375691,
 0.7126713974273761,
 0.7080759823080773,
 0.7052810842125017,
 0.7017390144459775,
 0.6973315110509178,
 0.6973315110509178,
 0.6940397558702132,
 0.6878969692409262,
 0.6856618618446203,
 0.6846435881667342,
 0.6820643170409181,
 0.6819455939719691,
 0.679511764972802,
 0.6775818498554959,
 0.6732918971639353,
 0.6695235571538055,
 0.669359726137524,
 0.6681720834706426,
 0.6645535358279497,
 0.6642252855583435,
 0.6629710590850502,
 0.6622818617649432,
 0.66114448310338,
 0.6598553489525154,
 0.6598553489525154,
 0.6584916224824324,
 0.6558896886235841,
 0.6547889516080795,
 0.6517426458680396,
 0.6494855902119164,
 0.648039739771364,
 0.6463374646359593,
 0.6408502710866866,
 0.5590107993479483,
 0.545606365524242,
 0.5404115378024161,
 0.5404115378024161,
 0.5404115378024161,
 0.5338765887633339,
 0.5317680439796522,
 0.528663668409196,
 0.518601383433624,
 0.5170635231056915,


In [96]:
df = df1[df1.index.isin(recommended_prods)]
df

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower
sku,,,,,,,,,,
9790153431,Manish Malhotra Gold Flake Lipgloss,Makeup,Lip Makeup,Lip Gloss,Manish Malhotra Gold Flake Lipgloss,1.0,Manish Malhotra,Manish Malhotra Gold Flake Lipgloss Lip Makeup...,mgp,manish malhotra gold flake lipgloss lip makeup...
8906095880140,Perfect Curves Matte Lip Crayon - Biscotti Old,Makeup,Lip Makeup,Matte Lipstick | Lipstick,Perfect Curves,795.0,MyGlamm,Perfect Curves Matte Lip Crayon - Biscotti Old...,mgp,perfect curves matte lip crayon - biscotti old...
8906095880157,Perfect Curves Matte Lip Crayon Old - Debutante,Makeup,Lip Makeup,Matte Lipstick | Lipstick,Perfect Curves,795.0,MyGlamm,Perfect Curves Matte Lip Crayon Old - Debutant...,mgp,perfect curves matte lip crayon old - debutant...
8906095880201,Perfect Curves Matte Lip Crayon Old - Soleil,Makeup,Lip Makeup,Matte Lipstick | Lipstick,Perfect Curves,795.0,MyGlamm,Perfect Curves Matte Lip Crayon Old - Soleil L...,mgp,perfect curves matte lip crayon old - soleil l...
8906095880904,Butterlicious Liquid Matte Lipstick - Possessed,Makeup,Lip Makeup,Lipstick,Butterlicious,895.0,MyGlamm,Butterlicious Liquid Matte Lipstick - Possesse...,mgp,butterlicious liquid matte lipstick - possesse...
...,...,...,...,...,...,...,...,...,...,...
8906095889341,MyGlamm Ultimatte Long Lasting Liquid Lipstick...,Makeup,Lip Makeup,Liquid Lipstick | Lipstick,Ultimatte Long Lasting Liquid Lipstick,599.0,MyGlamm,MyGlamm Ultimatte Long Lasting Liquid Lipstick...,mgp,myglamm ultimatte long lasting liquid lipstick...
8906095889730,Manish Malhotra Beauty Lip Liner & Filler - F...,Makeup,Lip Makeup,Lip Liner,Manish Malhotra Lip Liner & Filler,950.0,Manish Malhotra,Manish Malhotra Beauty Lip Liner & Filler - F...,mgp,manish malhotra beauty lip liner & filler - f...
8906095889747,Manish Malhotra Beauty Lip Liner & Filler - B...,Makeup,Lip Makeup,Lip Liner,Manish Malhotra Lip Liner & Filler,950.0,Manish Malhotra,Manish Malhotra Beauty Lip Liner & Filler - B...,mgp,manish malhotra beauty lip liner & filler - b...


In [97]:
df = df.reindex(recommended_prods)
df

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower
sku,,,,,,,,,,
8906095888832,Manish Malhotra Liquid Matte Lipstick - Muse,Makeup,Lip Makeup,Lipstick | Matte Lipstick | Liquid Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Muse L...,mgp,manish malhotra liquid matte lipstick - muse l...
8906095888849,Manish Malhotra Liquid Matte Lipstick - Barely...,Makeup,Lip Makeup,Matte Lipstick | Liquid Lipstick | Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Barely...,mgp,manish malhotra liquid matte lipstick - barely...
8906095888795,Manish Malhotra Liquid Matte Lipstick - Obsession,Makeup,Lip Makeup,Lipstick | Matte Lipstick | Liquid Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Obsess...,mgp,manish malhotra liquid matte lipstick - obsess...
8906095883592,Manish Malhotra Liquid Matte Lipstick - Eyes O...,Makeup,Lip Makeup,Lipstick | Liquid Lipstick | Matte Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Eyes O...,mgp,manish malhotra liquid matte lipstick - eyes o...
8906095883523,Manish Malhotra Liquid Matte Lipstick - Mess W...,Makeup,Lip Makeup,Lipstick | Liquid Lipstick | Matte Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Mess W...,mgp,manish malhotra liquid matte lipstick - mess w...
...,...,...,...,...,...,...,...,...,...,...
8906095884124,MyGlamm K.Play Flavoured Lipgloss - Orange Spin,Makeup,Lip Makeup,Lip Gloss,K.Play Flavoured Lipgloss,595.0,K.Play,MyGlamm K.Play Flavoured Lipgloss - Orange Spi...,mgp,myglamm k.play flavoured lipgloss - orange spi...
8906095884094,MyGlamm K.Play Flavoured Lipgloss - Mango Swirl,Makeup,Lip Makeup,Lip Gloss,K.Play Flavoured Lipgloss,595.0,K.Play,MyGlamm K.Play Flavoured Lipgloss - Mango Swir...,mgp,myglamm k.play flavoured lipgloss - mango swir...
8906095884025,MyGlamm K.Play Flavoured Lipgloss - Apple Crunch,Makeup,Lip Makeup,Lip Gloss,K.Play Flavoured Lipgloss,595.0,K.Play,MyGlamm K.Play Flavoured Lipgloss - Apple Crun...,mgp,myglamm k.play flavoured lipgloss - apple crun...


In [98]:
df['score'] = recommeded_scores
df.score.describe()

count    370.000000
mean       0.324807
std        0.161383
min        0.050094
25%        0.243958
50%        0.312180
75%        0.371854
max        0.761270
Name: score, dtype: float64

In [34]:
df=df[df['score']>0.1]
df

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower,score
sku,,,,,,,,,,,
8906095888832,Manish Malhotra Liquid Matte Lipstick - Muse,Makeup,Lip Makeup,Lipstick | Matte Lipstick | Liquid Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Muse L...,mgp,manish malhotra liquid matte lipstick - muse l...,0.761565
8906095888849,Manish Malhotra Liquid Matte Lipstick - Barely...,Makeup,Lip Makeup,Matte Lipstick | Liquid Lipstick | Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Barely...,mgp,manish malhotra liquid matte lipstick - barely...,0.743996
8906095888795,Manish Malhotra Liquid Matte Lipstick - Obsession,Makeup,Lip Makeup,Lipstick | Matte Lipstick | Liquid Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Obsess...,mgp,manish malhotra liquid matte lipstick - obsess...,0.733439
8906095883592,Manish Malhotra Liquid Matte Lipstick - Eyes O...,Makeup,Lip Makeup,Lipstick | Liquid Lipstick | Matte Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Eyes O...,mgp,manish malhotra liquid matte lipstick - eyes o...,0.721709
8906095883523,Manish Malhotra Liquid Matte Lipstick - Mess W...,Makeup,Lip Makeup,Lipstick | Liquid Lipstick | Matte Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Mess W...,mgp,manish malhotra liquid matte lipstick - mess w...,0.715542
...,...,...,...,...,...,...,...,...,...,...,...
8906095886548,MyGlamm LIT Matte Lipliner Pencil - Woke,Makeup,Lip Makeup,Lip Liner,LIT Matte Lipliner Pencil,445.0,LIT,MyGlamm LIT Matte Lipliner Pencil - Woke Lip M...,mgp,myglamm lit matte lipliner pencil - woke lip m...,0.125401
8906095886500,MyGlamm LIT Matte Lipliner Pencil - Vsco,Makeup,Lip Makeup,Lip Liner,LIT Matte Lipliner Pencil,445.0,LIT,MyGlamm LIT Matte Lipliner Pencil - Vsco Lip M...,mgp,myglamm lit matte lipliner pencil - vsco lip m...,0.123203
8906095886517,MyGlamm LIT Matte Lipliner Pencil - Yeet,Makeup,Lip Makeup,Lip Liner,LIT Matte Lipliner Pencil,445.0,LIT,MyGlamm LIT Matte Lipliner Pencil - Yeet Lip M...,mgp,myglamm lit matte lipliner pencil - yeet lip m...,0.123203


In [35]:
df = df.sort_values(by=['score','base_price'],ascending=False)
df

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower,score
sku,,,,,,,,,,,
8906095888832,Manish Malhotra Liquid Matte Lipstick - Muse,Makeup,Lip Makeup,Lipstick | Matte Lipstick | Liquid Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Muse L...,mgp,manish malhotra liquid matte lipstick - muse l...,0.761565
8906095888849,Manish Malhotra Liquid Matte Lipstick - Barely...,Makeup,Lip Makeup,Matte Lipstick | Liquid Lipstick | Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Barely...,mgp,manish malhotra liquid matte lipstick - barely...,0.743996
8906095888795,Manish Malhotra Liquid Matte Lipstick - Obsession,Makeup,Lip Makeup,Lipstick | Matte Lipstick | Liquid Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Obsess...,mgp,manish malhotra liquid matte lipstick - obsess...,0.733439
8906095883592,Manish Malhotra Liquid Matte Lipstick - Eyes O...,Makeup,Lip Makeup,Lipstick | Liquid Lipstick | Matte Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Eyes O...,mgp,manish malhotra liquid matte lipstick - eyes o...,0.721709
8906095883523,Manish Malhotra Liquid Matte Lipstick - Mess W...,Makeup,Lip Makeup,Lipstick | Liquid Lipstick | Matte Lipstick,Manish Malhotra Liquid Matte Lipstick,995.0,Manish Malhotra,Manish Malhotra Liquid Matte Lipstick - Mess W...,mgp,manish malhotra liquid matte lipstick - mess w...,0.715542
...,...,...,...,...,...,...,...,...,...,...,...
8906095886548,MyGlamm LIT Matte Lipliner Pencil - Woke,Makeup,Lip Makeup,Lip Liner,LIT Matte Lipliner Pencil,445.0,LIT,MyGlamm LIT Matte Lipliner Pencil - Woke Lip M...,mgp,myglamm lit matte lipliner pencil - woke lip m...,0.125401
8906095886500,MyGlamm LIT Matte Lipliner Pencil - Vsco,Makeup,Lip Makeup,Lip Liner,LIT Matte Lipliner Pencil,445.0,LIT,MyGlamm LIT Matte Lipliner Pencil - Vsco Lip M...,mgp,myglamm lit matte lipliner pencil - vsco lip m...,0.123203
8906095886517,MyGlamm LIT Matte Lipliner Pencil - Yeet,Makeup,Lip Makeup,Lip Liner,LIT Matte Lipliner Pencil,445.0,LIT,MyGlamm LIT Matte Lipliner Pencil - Yeet Lip M...,mgp,myglamm lit matte lipliner pencil - yeet lip m...,0.123203


In [36]:
recommeded_order_sku = df.index
recommeded_order_sku

Index(['8906095888832', '8906095888849', '8906095888795', '8906095883592',
       '8906095883523', '8906095882922', '8906095882892', '8906095883608',
       '8906095888825', '8906095883547',
       ...
       '8906095883974', '8906095883905', '8906095883998', '8906095883936',
       '8906095886524', '8906095886548', '8906095886500', '8906095886517',
       '8906095886531', '8906095887767'],
      dtype='object', name='sku', length=342)

In [37]:
recommeded_order_sku=[str(x) for x in recommeded_order_sku if x in (normal_sku)]
recommeded_order_sku

['8906095888832',
 '8906095888849',
 '8906095888795',
 '8906095883592',
 '8906095883523',
 '8906095882922',
 '8906095882892',
 '8906095883608',
 '8906095888825',
 '8906095883547',
 '8906095883585',
 '8906095885190',
 '8906095888856',
 '8906095882854',
 '8906095882960',
 '8906095883561',
 '8906095882878',
 '8906095888818',
 '8906095885206',
 '8906095883554',
 '8906095882977',
 '8906095882908',
 '8906095885213',
 '8906095882915',
 '8906095882939',
 '8906095883615',
 '8906095888801',
 '8906095882885',
 '8906095882953',
 '8906095882946',
 '8906095883530',
 '8906095883622',
 '8906095888863',
 '8906095882847',
 '8906095883578',
 '8906095882984',
 '8906095883516',
 '8906095882861',
 '8906095885237',
 '8906095888757',
 '8906095885244',
 '8906095883455',
 '8906095888764',
 '8906095885220',
 '8906095881772',
 '8906095883448',
 '8906095888740',
 '8906095883424',
 '8906095888788',
 '8906095881796',
 '8906095881819',
 '8906095881826',
 '8906095888771',
 '8906095881833',
 '8906095881840',
 '89060958

In [38]:
recomm_skus=df_product_category[df_product_category['sku'].isin(recommeded_order_sku)]
recomm_skus

,sku,category_name,product_name
1,8906095881758,Manish Malhotra Hi-Shine Lipstick,Manish Malhotra Hi-Shine Lipstick - Desert Suede
4,8906095882984,Manish Malhotra Soft Matte Lipstick,Manish Malhotra Soft Matte Lipstick - Burnt Rust
5,8906095883257,POSE HD Lipstick,MyGlamm POSE HD Lipstick - Muted Brown
6,8906095883813,LIT Liquid Matte Lipstick,MyGlamm LIT Liquid Matte Lipstick - Slow Fade
7,8906095883790,LIT Liquid Matte Lipstick,MyGlamm LIT Liquid Matte Lipstick - Thirst Trap
...,...,...,...
864,8906095883516,Manish Malhotra Liquid Matte Lipstick,Manish Malhotra Liquid Matte Lipstick - Carame...
867,8906095885831,LIT Liquid Matte Lipstick,MyGlamm LIT Liquid Matte Lipstick - Instagrand...
869,8906095886210,LIT Velvet Matte Liquid Lipstick,MyGlamm LIT Velvet Matte Liquid Lipstick - Tha...
870,8906095886197,LIT Velvet Matte Liquid Lipstick,MyGlamm LIT Velvet Matte Liquid Lipstick - Yummy


In [39]:
recomm_skus1=recomm_skus.drop_duplicates(subset=['category_name'], keep='first')
recomm_skus1

,sku,category_name,product_name
1,8906095881758,Manish Malhotra Hi-Shine Lipstick,Manish Malhotra Hi-Shine Lipstick - Desert Suede
4,8906095882984,Manish Malhotra Soft Matte Lipstick,Manish Malhotra Soft Matte Lipstick - Burnt Rust
5,8906095883257,POSE HD Lipstick,MyGlamm POSE HD Lipstick - Muted Brown
6,8906095883813,LIT Liquid Matte Lipstick,MyGlamm LIT Liquid Matte Lipstick - Slow Fade
20,8906095885183,Perfect Curves,Perfect Curves Matte Lip Crayon - Candy
31,8906095889228,Ultimatte Long Lasting Liquid Lipstick,MyGlamm Ultimatte Long Lasting Liquid Lipstick...
41,8906095883950,K.Play Flavoured Lipstick,MyGlamm K.Play Flavoured Lipstick - Berry Blast
48,8906095886913,Ultimatte Long-Stay Matte Lipstick,Ultimatte Long-Stay Matte Lipstick - Goddess
52,8906095885541,LIT Creamy Matte Lipstick,MyGlamm LIT Creamy Matte Lipstick - LIIT
67,8906095888818,Manish Malhotra Liquid Matte Lipstick,Manish Malhotra Liquid Matte Lipstick - Ladies...


In [40]:
recomm_skus2=recomm_skus.drop_duplicates(subset=['category_name'], keep='first')['product_name'].values
recomm_skus2[:20]

array(['Manish Malhotra Hi-Shine Lipstick - Desert Suede',
       'Manish Malhotra Soft Matte Lipstick - Burnt Rust',
       'MyGlamm POSE HD Lipstick - Muted Brown',
       'MyGlamm LIT Liquid Matte Lipstick - Slow Fade',
       'Perfect Curves Matte Lip Crayon - Candy',
       'MyGlamm Ultimatte Long Lasting Liquid Lipstick - Plum Beauty',
       'MyGlamm K.Play Flavoured Lipstick - Berry Blast',
       'Ultimatte Long-Stay Matte Lipstick - Goddess',
       'MyGlamm LIT Creamy Matte Lipstick - LIIT',
       'Manish Malhotra Liquid Matte Lipstick - Ladies Night',
       'Manish Malhotra Hi-Shine Lipgloss - Retro Romance',
       'MyGlamm LIT Satin Matte Lipstick - The Affair',
       'POPxo Makeup - Pretty Mess - Mini Lip Kit',
       'MyGlamm LIT Velvet Matte Liquid Lipstick - IDGAF',
       'MyGlamm LIT Matte Lipliner Pencil - Blended'], dtype=object)

In [41]:
def remove_same_category_products(x):
    recomm_skus=df_product_category[df_product_category['sku'].isin(x)]
    sku_name = df_product_category[df_product_category['sku'].isin(x)]
#     recomm_skus=recomm_skus.drop_duplicates(subset=['category_name'], keep='first')['sku'].values
    product_name=recomm_skus.drop_duplicates(subset=['category_name'], keep='first')['product_name'].values
    return product_name

In [42]:
def recommendations(name, remove_filter, cosine_similarities = cosine_similarities):
    global recommended_prods
    recommended_prods = []
    recommended_indices = []
    recommeded_scores = []
    recommeded_order_sku = []
    # gettin the index of the product that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    # getting the indexes of all products 
    all_indexes = list(score_series.index)
    
    for i in all_indexes:
        if df1['product'][idx] != df1['product'][i]:
            if df1[remove_filter][idx] == df1[remove_filter][i]:
                recommended_prods.append((df1.index)[i])
                recommended_indices.append(i)
                recommeded_scores.append(score_series[i])
                #j = j + 1
                
    df = df1[df1.index.isin(recommended_prods)]
    df = df.reindex(recommended_prods)
    df['score'] = recommeded_scores
    df=df[df['score']>0.05]
    df = df.sort_values(by=['score','base_price'],ascending=False)
    
    recommeded_order_sku = df.index
    
    recommeded_order_sku=[str(x) for x in recommeded_order_sku if x in (normal_sku)]
    # print(recommeded_order_sku)
        
    return recommeded_order_sku

In [43]:
df1['recommended_sku_name'] = np.empty((len(df1), 0)).tolist()
df1['recommended_sku_name']

sku
8076149815          []
9790153431          []
80761498143         []
STPL1707001         []
8906095887965       []
                    ..
sgl8906095880416    []
sgl8906095880423    []
sgl8906095880430    []
sgl8906095880447    []
sgl8906095880454    []
Name: recommended_sku_name, Length: 875, dtype: object

In [44]:
for i in range(0, len(indices)):
    
    recommeded_order_sku = recommendations(indices[i],'category')
    df1['recommended_sku_name'][i] = remove_same_category_products(recommeded_order_sku)[:20]
    

In [45]:
df1['len']=[ len(x) for x in df1['recommended_sku_name'].values]
df1

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower,recommended_sku_name,len
sku,,,,,,,,,,,,
8076149815,Manish Malhotra Powder Matte Lipstick,Makeup,Lip Makeup,Matte Lipstick,Manish Malhotra Powder Matte Lipstick,1.0,Manish Malhotra,Manish Malhotra Powder Matte Lipstick Lip Make...,mgp,manish malhotra powder matte lipstick lip make...,[Manish Malhotra Hi-Shine Lipstick - Desert Su...,19
9790153431,Manish Malhotra Gold Flake Lipgloss,Makeup,Lip Makeup,Lip Gloss,Manish Malhotra Gold Flake Lipgloss,1.0,Manish Malhotra,Manish Malhotra Gold Flake Lipgloss Lip Makeup...,mgp,manish malhotra gold flake lipgloss lip makeup...,[Manish Malhotra Hi-Shine Lipstick - Desert Su...,16
80761498143,Manish Malhotra Strobing Cream,Makeup,Face Makeup,Highlighter,Manish Malhotra Strobing Cream,1.0,Manish Malhotra,Manish Malhotra Strobing Cream Face Makeup Hig...,mgp,manish malhotra strobing cream face makeup hig...,[Total Makeover FF Cream Foundation Palette - ...,15
STPL1707001,MyGlamm Vanity Box,Makeup,Makeup Brushes And Accessories,Accessories,MyGlamm Vanity Box,2000.0,MyGlamm,MyGlamm Vanity Box Makeup Brushes And Accessor...,mgp,myglamm vanity box makeup brushes and accessor...,"[Eye Shader Brush, Sling Bag - Black, Beauty B...",15
8906095887965,MyGlamm Avacado & Aloe Eye Cream - 30 Gm,Skincare,Eye Care,Eye Cream,Avacado & Aloe Eye Cream,499.0,MyGlamm,MyGlamm Avacado & Aloe Eye Cream - 30 Gm Eye C...,mgp,myglamm avacado & aloe eye cream - 30 gm eye c...,"[GLOW Iridescent Brightening Eye Cream, MyGlam...",2
...,...,...,...,...,...,...,...,...,...,...,...,...
sgl8906095880416,TWO OF YOUR KIND -Ichor,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Ichor Nail Polish Sets Nail ...,mgp,two of your kind -ichor nail polish sets nail ...,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG...",8
sgl8906095880423,TWO OF YOUR KIND -True Blue,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -True Blue Nail Polish Sets N...,mgp,two of your kind -true blue nail polish sets n...,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG...",8
sgl8906095880430,TWO OF YOUR KIND -Oh My Gold,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Oh My Gold Nail Polish Sets ...,mgp,two of your kind -oh my gold nail polish sets ...,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG...",8


In [46]:
df1.reset_index(inplace=True)

In [47]:
indexes=df1[df1['len']==0].index

In [48]:
df1.set_index('sku',inplace=True)

In [49]:
for i in range(0, len(indices)):
    
    recommeded_order_sku = recommendations(indices[i],'category')
    df1['recommended_sku_name'][i] = remove_same_category_products(recommeded_order_sku)[:20]
    

In [50]:
df1

,sku_name,parent_category,category,sub_category,product,base_price,brand_name,Description,vendor_code,lower,recommended_sku_name,len
sku,,,,,,,,,,,,
8076149815,Manish Malhotra Powder Matte Lipstick,Makeup,Lip Makeup,Matte Lipstick,Manish Malhotra Powder Matte Lipstick,1.0,Manish Malhotra,Manish Malhotra Powder Matte Lipstick Lip Make...,mgp,manish malhotra powder matte lipstick lip make...,[Manish Malhotra Hi-Shine Lipstick - Desert Su...,19
9790153431,Manish Malhotra Gold Flake Lipgloss,Makeup,Lip Makeup,Lip Gloss,Manish Malhotra Gold Flake Lipgloss,1.0,Manish Malhotra,Manish Malhotra Gold Flake Lipgloss Lip Makeup...,mgp,manish malhotra gold flake lipgloss lip makeup...,[Manish Malhotra Hi-Shine Lipstick - Desert Su...,16
80761498143,Manish Malhotra Strobing Cream,Makeup,Face Makeup,Highlighter,Manish Malhotra Strobing Cream,1.0,Manish Malhotra,Manish Malhotra Strobing Cream Face Makeup Hig...,mgp,manish malhotra strobing cream face makeup hig...,[Total Makeover FF Cream Foundation Palette - ...,15
STPL1707001,MyGlamm Vanity Box,Makeup,Makeup Brushes And Accessories,Accessories,MyGlamm Vanity Box,2000.0,MyGlamm,MyGlamm Vanity Box Makeup Brushes And Accessor...,mgp,myglamm vanity box makeup brushes and accessor...,"[Eye Shader Brush, Sling Bag - Black, Beauty B...",15
8906095887965,MyGlamm Avacado & Aloe Eye Cream - 30 Gm,Skincare,Eye Care,Eye Cream,Avacado & Aloe Eye Cream,499.0,MyGlamm,MyGlamm Avacado & Aloe Eye Cream - 30 Gm Eye C...,mgp,myglamm avacado & aloe eye cream - 30 gm eye c...,"[GLOW Iridescent Brightening Eye Cream, MyGlam...",2
...,...,...,...,...,...,...,...,...,...,...,...,...
sgl8906095880416,TWO OF YOUR KIND -Ichor,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Ichor Nail Polish Sets Nail ...,mgp,two of your kind -ichor nail polish sets nail ...,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG...",8
sgl8906095880423,TWO OF YOUR KIND -True Blue,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -True Blue Nail Polish Sets N...,mgp,two of your kind -true blue nail polish sets n...,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG...",8
sgl8906095880430,TWO OF YOUR KIND -Oh My Gold,Makeup,Nail Polish Sets,Nail Polish,Two of Your Kind,195.0,MyGlamm,TWO OF YOUR KIND -Oh My Gold Nail Polish Sets ...,mgp,two of your kind -oh my gold nail polish sets ...,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG...",8


In [51]:
df2=df1[['recommended_sku_name']].reset_index()
df2

,sku,recommended_sku_name
0,8076149815,[Manish Malhotra Hi-Shine Lipstick - Desert Su...
1,9790153431,[Manish Malhotra Hi-Shine Lipstick - Desert Su...
2,80761498143,[Total Makeover FF Cream Foundation Palette - ...
3,STPL1707001,"[Eye Shader Brush, Sling Bag - Black, Beauty B..."
4,8906095887965,"[GLOW Iridescent Brightening Eye Cream, MyGlam..."
...,...,...
870,sgl8906095880416,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG..."
871,sgl8906095880423,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG..."
872,sgl8906095880430,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG..."
873,sgl8906095880447,"[Manish Malhotra Nail Lacquer - Teal Tale, MyG..."


In [52]:
df1[['sku_name','recommended_sku_name']].to_csv("recommended_sku_name.csv")

In [53]:
#len(df1[df1['recommended_sku']==""])

In [54]:
# prepare products details dictionary with sku as key and details as its value as another dictionary

product_details_query = '''
select distinct t1.*, t2.totalCount, t2.avgRating from (
select 
    p.sku,
    p.brand_name,
    p.parent_category_name,
    p.category_name,
    p.product_name,
    p.product_url,
    p.base_price as priceMRP,
    p.offer_price as priceOffer,
    p.product_url as url,
    p.shadelabel as shade,
    c.productTitle,
    c.productSubTitle,
    c.productName,
    s.slug,
    imageURL,
    videoURL,
    pp.category_rank
from 
    `myglamm-india.master_tables.master_product_table_v2` p
left outer join (
    select 
        sourceid as product_id,
        data.metadata.title as productTitle,
        data.content.subtitle as productSubTitle,
        data.content.name as productName
    from 
        `myglamm-india.production_mongo_v2.Content`, unnest(properties.filter) as pf
    where   
        source = 'product' 
        and pf.value.language = 'EN'
) c on p.product_id = c.product_id
left outer join (
    select 
        sourceid as product_id,
        url as slug
    from    
        `myglamm-india.production_mongo_v2.Url`
    where 
        source = 'product' 
) s on p.product_id = s.product_id
left outer join (
    select 
        product_id, 
        imageURL 
    from (
        select 
            _id,
            sourceid as product_id,
            imageurl._400x400 as imageURL,
            displayorder,
            RANK() OVER ( PARTITION BY sourceid ORDER BY displayorder ) AS rank
        from 
            `myglamm-india.production_mongo_v2.asset`
        where 
            source = 'product' 
            and statusid = 1 
            and type = 'image'
        order by 
            displayOrder asc
    ) 
    where 
        rank = 1
) i on p.product_id = i.product_id
left outer join (
    select 
        product_id, 
        videoURL 
    from (
        select 
            _id,
            sourceid as product_id,
            url as videoURL,
            displayorder,
            RANK() OVER ( PARTITION BY sourceid ORDER BY displayorder ) AS rank
        from 
            `myglamm-india.production_mongo_v2.asset`
        where 
            source = 'product' 
            and statusid = 1 
            and type = 'video'
        order by 
            displayOrder asc
    ) where rank = 1
) v on p.product_id = v.product_id
left outer join 
    `myglamm-india.mg_reports_primary.product_persona` pp on p.sku = pp.sku
where 
    p.product_status = 'Active' and p.vendor_code='mgp'
) t1   
left join (
select * from (
SELECT distinct
        case
            when meta.sku is null and meta.sku__st is null and reviewerinfo.additional.sku is not null then cast(reviewerinfo.additional.sku as string)
            when meta.sku is null then meta.sku__st
            else cast(meta.sku as string)
        end as sku,
        meta.productname,
        count(*) as totalCount,
        round(avg(rating),2) as avgRating
    FROM
         `myglamm-india.authorized_views.reviews_av` 
    group by
        1, 2
) where sku is not null
) t2 on t1.sku=t2.sku and t1.product_name=t2.productname
where t1.sku not in (
    select distinct sku from (
        SELECT 
          sku 
        FROM `myglamm-india.production_mongo_v2.Products` 
        where productmeta.isessential=true

        union all

        SELECT  
          sku
        FROM `myglamm-india.production_mongo_v2.comboProduct` 
        where productmeta.isessential=true

        union all
         
        select sku from `myglamm-india.master_tables.master_product_table_v2`  where sku not in (SELECT sku FROM `myglamm-india.master_tables.master_product_table_v2` 
        where hidden=true 
        or in_stock=true
        )

    )
)
order by 
    t1.category_rank
    
'''

In [55]:
product_details_job = client.query(product_details_query)
product_details_job

QueryJob<project=myglamm-india, location=asia-south1, id=be2d2887-4968-486d-a7d1-a7218c592962>

In [56]:
product_details_json = product_details_job.result()
product_details_json

In [57]:
product_details_json_dict = {}

In [58]:
for row in product_details_json:
    # print(row)
    product_details_json_dict[row['sku']] = {
        'sku': row['sku'],
        'imageURL': row['imageURL'],
        'videoURL': row['videoURL'],
        'productName': row['productName'],
        'productTitle': row['productTitle'],
        'productSubTitle': row['productSubTitle'],
        'slug': row['slug'],
        'priceMRP': row['priceMRP'],
        'priceOffer': row['priceOffer'],
        'url': row['url'],
        'shade': row['shade'],
        'totalCount': row['totalCount'],
        'avgRating': row['avgRating']
    }

print(product_details_json_dict)

{'combo2069': {'sku': 'combo2069', 'imageURL': None, 'videoURL': None, 'productName': None, 'productTitle': None, 'productSubTitle': None, 'slug': None, 'priceMRP': 543.0, 'priceOffer': 543.0, 'url': 'https://myglamm.in/hpS4B0dTRlb', 'shade': None, 'totalCount': None, 'avgRating': None}, 'CMB00069': {'sku': 'CMB00069', 'imageURL': None, 'videoURL': None, 'productName': None, 'productTitle': None, 'productSubTitle': None, 'slug': None, 'priceMRP': 159.0, 'priceOffer': 159.0, 'url': 'https://myglamm.in/UIq7i9bJSnb', 'shade': None, 'totalCount': None, 'avgRating': None}, 'STPL-1844-001': {'sku': 'STPL-1844-001', 'imageURL': 'https://files.myglamm.com/site-images/400x400/STPL-1844-001.jpg', 'videoURL': None, 'productName': 'Folding Carry bag', 'productTitle': '', 'productSubTitle': 'Folding Carry bag', 'slug': '/product/folding-carry-bag.html', 'priceMRP': 45.0, 'priceOffer': 45.0, 'url': 'https://myglamm.in/MT7NXvo5anb', 'shade': None, 'totalCount': None, 'avgRating': None}, '890609588933

In [59]:
similar_products = {}

for x in df2.iterrows():
    sku = x[1][0]
    recommended_sku = x[1][1]

    print('sku')
    print(sku)
    print('recommended_sku')
    print(recommended_sku)

    similar_products[sku] = []
    for rsku in recommended_sku:
        print('sku')
        print(sku)
        print('rsku')
        print(rsku)
        if rsku in product_details_json_dict:
            product_details_json_dict[rsku]
            smprod = {
                "sku":product_details_json_dict[rsku]['sku'],
                "imageURL":product_details_json_dict[rsku]['imageURL'],
                "videoURL":product_details_json_dict[rsku]['videoURL'],
                "productName":product_details_json_dict[rsku]['productName'],
                "productTitle":product_details_json_dict[rsku]['productTitle'],
                "productSubTitle":product_details_json_dict[rsku]['productSubTitle'],
                "slug":product_details_json_dict[rsku]['slug'],
                "priceMRP":product_details_json_dict[rsku]['priceMRP'],
                "priceOffer":product_details_json_dict[rsku]['priceOffer'],
                "url":product_details_json_dict[rsku]['url'],
                "shade":product_details_json_dict[rsku]['shade']
            }

            if 'totalCount' in  product_details_json_dict[rsku] and 'avgRating' in product_details_json_dict[rsku] and product_details_json_dict[rsku]['totalCount'] != None and product_details_json_dict[rsku]['avgRating'] != None:
                smprod['rating'] = {
                    "totalCount": product_details_json_dict[rsku]['totalCount'],
                    "avgRating": product_details_json_dict[rsku]['avgRating']
                }
            
            similar_products[sku].append(smprod)



sku
8076149815
recommended_sku
['Manish Malhotra Hi-Shine Lipstick - Desert Suede'
 'Manish Malhotra Soft Matte Lipstick - Burnt Rust'
 'MyGlamm POSE HD Lipstick - Muted Brown'
 'MyGlamm LIT Liquid Matte Lipstick - Slow Fade'
 'Perfect Curves Matte Lip Crayon - Candy'
 'MyGlamm Ultimatte Long Lasting Liquid Lipstick - Plum Beauty'
 'MyGlamm K.Play Flavoured Lipstick - Berry Blast'
 'Ultimatte Long-Stay Matte Lipstick - Goddess'
 'MyGlamm LIT Creamy Matte Lipstick - LIIT'
 'MyGlamm K.Play Flavoured Lipgloss - Cherry Burst'
 'Manish Malhotra Liquid Matte Lipstick - Ladies Night'
 'MyGlamm SUPERFOODS Color Pop Lip Balm - Berry'
 'Manish Malhotra Hi-Shine Lipgloss - Retro Romance'
 'MyGlamm LIT Satin Matte Lipstick - The Affair'
 'MyGlamm LIT - pH Lip Balm - Orange Crush'
 'POPxo Makeup - Pretty Mess - Mini Lip Kit'
 'MyGlamm LIT Velvet Matte Liquid Lipstick - IDGAF'
 'MyGlamm LIT Matte Lipliner Pencil - Blended'
 'Lip Volumizer Instant Mega Plumping Lip Gloss']
sku
8076149815
rsku
Manish 

rsku
Makeup Blender
sku
STPL1703001
rsku
Magic Sharpener
sku
STPL1703001
rsku
Flat Concealer Brush
sku
STPL1703001
rsku
White Pouch With Golden Border
sku
STPL1703001
rsku
Concealer Brush For Total Makeover FF Cream
sku
STPL1703001
rsku
Eye Blender Brush
sku
STPL1703001
rsku
Trendy Pouch - Pink
sku
STPL1703001
rsku
Lip Balm Pouch
sku
STPL1703001
rsku
Eye Tapered Blender Brush
sku
STPL1703001
rsku
Myglamm Superfoods Pouch
sku
STPL1703001
rsku
Powder Brush
sku
STPL1703001
rsku
Trendy Pouch - Green
sku
STPL1704001
recommended_sku
['Eye Shader Brush' 'Sling Bag - Black' 'Beauty Blender' 'Makeup Blender'
 'Magic Sharpener' 'Flat Concealer Brush' 'White Pouch With Golden Border'
 'Concealer Brush For Total Makeover FF Cream' 'Eye Blender Brush'
 'Trendy Pouch - Pink' 'Lip Balm Pouch' 'Eye Tapered Blender Brush'
 'Myglamm Superfoods Pouch' 'Powder Brush' 'Trendy Pouch - Green']
sku
STPL1704001
rsku
Eye Shader Brush
sku
STPL1704001
rsku
Sling Bag - Black
sku
STPL1704001
rsku
Beauty Blender
sku

MyGlamm LIT Matte Nail Enamel - Salty
sku
8906095882267
rsku
Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique
sku
8906095882267
rsku
Wanderlust Chrome Matte Nail Polish - Aruba
sku
8906095882267
rsku
Wanderlust Sand Matte Nail Polish - Raspberry Trifle
sku
8906095882267
rsku
Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush
sku
8906095882267
rsku
MyGlamm LIT Nail Enamel - Boi Bye
sku
8906095882267
rsku
POPxo Makeup - Dreamin’ Mini Nail Kit
sku
8906095882267
rsku
MyGlamm Two of your kind Nail Enamel Set - Casino Night
sku
8906095882274
recommended_sku
['Manish Malhotra Nail Lacquer - Teal Tale'
 'MyGlamm LIT Matte Nail Enamel - Salty'
 'Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique'
 'Wanderlust Chrome Matte Nail Polish - Aruba'
 'Wanderlust Sand Matte Nail Polish - Raspberry Trifle'
 'Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush'
 'MyGlamm LIT Nail Enamel - Boi Bye'
 'POPxo Makeup - Dreamin’ Mini Nail Kit'
 'MyGlamm Two of

Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush
sku
8906095882373
rsku
MyGlamm LIT Nail Enamel - Boi Bye
sku
8906095882373
rsku
POPxo Makeup - Dreamin’ Mini Nail Kit
sku
8906095882373
rsku
MyGlamm Two of your kind Nail Enamel Set - Casino Night
sku
8906095882380
recommended_sku
['Manish Malhotra Nail Lacquer - Teal Tale'
 'MyGlamm LIT Matte Nail Enamel - Salty'
 'Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique'
 'Wanderlust Chrome Matte Nail Polish - Aruba'
 'Wanderlust Sand Matte Nail Polish - Raspberry Trifle'
 'Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush'
 'MyGlamm LIT Nail Enamel - Boi Bye'
 'POPxo Makeup - Dreamin’ Mini Nail Kit'
 'MyGlamm Two of your kind Nail Enamel Set - Casino Night']
sku
8906095882380
rsku
Manish Malhotra Nail Lacquer - Teal Tale
sku
8906095882380
rsku
MyGlamm LIT Matte Nail Enamel - Salty
sku
8906095882380
rsku
Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique
sku
8906095882380
rsku
Wanderlust C

Wanderlust Sand Matte Nail Polish - Raspberry Trifle
sku
8906095885329
rsku
Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush
sku
8906095885329
rsku
MyGlamm LIT Nail Enamel - Boi Bye
sku
8906095885329
rsku
POPxo Makeup - Dreamin’ Mini Nail Kit
sku
8906095885329
rsku
MyGlamm Two of your kind Nail Enamel Set - Casino Night
sku
8906095885336
recommended_sku
['Manish Malhotra Nail Lacquer - Teal Tale'
 'MyGlamm LIT Matte Nail Enamel - Salty'
 'Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique'
 'Wanderlust Chrome Matte Nail Polish - Aruba'
 'Wanderlust Sand Matte Nail Polish - Raspberry Trifle'
 'Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush'
 'MyGlamm LIT Nail Enamel - Boi Bye'
 'POPxo Makeup - Dreamin’ Mini Nail Kit'
 'MyGlamm Two of your kind Nail Enamel Set - Casino Night']
sku
8906095885336
rsku
Manish Malhotra Nail Lacquer - Teal Tale
sku
8906095885336
rsku
MyGlamm LIT Matte Nail Enamel - Salty
sku
8906095885336
rsku
Manish Malhotra

...

sku
8906095889310
rsku
MyGlamm LIT Matte Lipliner Pencil - Blended
sku
8906095889310
rsku
Lip Volumizer Instant Mega Plumping Lip Gloss
sku
8906095889327
recommended_sku
['Manish Malhotra Hi-Shine Lipstick - Desert Suede'
 'Manish Malhotra Soft Matte Lipstick - Burnt Rust'
 'MyGlamm POSE HD Lipstick - Muted Brown'
 'MyGlamm LIT Liquid Matte Lipstick - Slow Fade'
 'Perfect Curves Matte Lip Crayon - Candy'
 'MyGlamm K.Play Flavoured Lipstick - Berry Blast'
 'Ultimatte Long-Stay Matte Lipstick - Goddess'
 'MyGlamm LIT Creamy Matte Lipstick - LIIT'
 'MyGlamm K.Play Flavoured Lipgloss - Cherry Burst'
 'Manish Malhotra Liquid Matte Lipstick - Ladies Night'
 'MyGlamm SUPERFOODS Color Pop Lip Balm - Berry'
 'Manish Malhotra Hi-Shine Lipgloss - Retro Romance'
 'MyGlamm LIT Satin Matte Lipstick - The Affair'
 'POPxo Makeup - Pretty Mess - Mini Lip Kit'
 'MyGlamm LIT Velvet Matte Liquid Lipstick - IDGAF'
 'MyGlamm LIT - pH Lip Balm - Bite Me'
 'MyGlamm LIT Matte Lipliner Pencil - Blended'
 'Lip V

rsku
MyGlamm Two of your kind Nail Enamel Set - Casino Night
sku
8906095889488
recommended_sku
['Manish Malhotra Nail Lacquer - Teal Tale'
 'MyGlamm LIT Matte Nail Enamel - Salty'
 'Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique'
 'Wanderlust Chrome Matte Nail Polish - Aruba'
 'Wanderlust Sand Matte Nail Polish - Raspberry Trifle'
 'MyGlamm LIT Nail Enamel - Boi Bye'
 'POPxo Makeup - Dreamin’ Mini Nail Kit'
 'MyGlamm Two of your kind Nail Enamel Set - Casino Night']
sku
8906095889488
rsku
Manish Malhotra Nail Lacquer - Teal Tale
sku
8906095889488
rsku
MyGlamm LIT Matte Nail Enamel - Salty
sku
8906095889488
rsku
Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique
sku
8906095889488
rsku
Wanderlust Chrome Matte Nail Polish - Aruba
sku
8906095889488
rsku
Wanderlust Sand Matte Nail Polish - Raspberry Trifle
sku
8906095889488
rsku
MyGlamm LIT Nail Enamel - Boi Bye
sku
8906095889488
rsku
POPxo Makeup - Dreamin’ Mini Nail Kit
sku
8906095889488
rsku
MyGlamm Two of your kind Nail En

sku
8906095889754
rsku
POPxo Makeup - Pretty Mess - Mini Lip Kit
sku
8906095889754
rsku
MyGlamm LIT Velvet Matte Liquid Lipstick - IDGAF
sku
8906095889754
rsku
MyGlamm LIT Matte Lipliner Pencil - Blended
sku
8906095889754
rsku
Lip Volumizer Instant Mega Plumping Lip Gloss
sku
8906095889761
recommended_sku
['Manish Malhotra Hi-Shine Lipstick - Desert Suede'
 'Manish Malhotra Soft Matte Lipstick - Burnt Rust'
 'MyGlamm LIT Liquid Matte Lipstick - Breadcrumb'
 'Perfect Curves Matte Lip Crayon - Candy'
 'MyGlamm Ultimatte Long Lasting Liquid Lipstick - Plum Beauty'
 'Ultimatte Long-Stay Matte Lipstick - Goddess'
 'MyGlamm LIT Creamy Matte Lipstick - LIIT'
 'MyGlamm K.Play Flavoured Lipgloss - Cherry Burst'
 'Manish Malhotra Liquid Matte Lipstick - Ladies Night'
 'MyGlamm SUPERFOODS Color Pop Lip Balm - Berry'
 'Manish Malhotra Hi-Shine Lipgloss - Retro Romance'
 'MyGlamm LIT - pH Lip Balm - Orange Crush'
 'POPxo Makeup - Pretty Mess - Mini Lip Kit'
 'MyGlamm LIT Satin Matte Lipstick - Dyna

Myglamm Superfoods Pouch
sku
STPL-1847-001
rsku
Powder Brush
sku
STPL-1847-001
rsku
Trendy Pouch - Green
sku
STPL-1855-001
recommended_sku
['Eye Shader Brush' 'Sling Bag - Black' 'Beauty Blender' 'Makeup Blender'
 'Magic Sharpener' 'Flat Concealer Brush' 'White Pouch With Golden Border'
 'Concealer Brush For Total Makeover FF Cream' 'Eye Blender Brush'
 'Trendy Pouch - Pink' 'Lip Balm Pouch' 'Eye Tapered Blender Brush'
 'Powder Brush' 'Trendy Pouch - Green']
sku
STPL-1855-001
rsku
Eye Shader Brush
sku
STPL-1855-001
rsku
Sling Bag - Black
sku
STPL-1855-001
rsku
Beauty Blender
sku
STPL-1855-001
rsku
Makeup Blender
sku
STPL-1855-001
rsku
Magic Sharpener
sku
STPL-1855-001
rsku
Flat Concealer Brush
sku
STPL-1855-001
rsku
White Pouch With Golden Border
sku
STPL-1855-001
rsku
Concealer Brush For Total Makeover FF Cream
sku
STPL-1855-001
rsku
Eye Blender Brush
sku
STPL-1855-001
rsku
Trendy Pouch - Pink
sku
STPL-1855-001
rsku
Lip Balm Pouch
sku
STPL-1855-001
rsku
Eye Tapered Blender Brush
sku
S

sgl8906095880430
rsku
Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush
sku
sgl8906095880430
rsku
MyGlamm LIT Nail Enamel - Boi Bye
sku
sgl8906095880430
rsku
POPxo Makeup - Dreamin’ Mini Nail Kit
sku
sgl8906095880447
recommended_sku
['Manish Malhotra Nail Lacquer - Teal Tale'
 'MyGlamm LIT Matte Nail Enamel - Salty'
 'Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique'
 'Wanderlust Chrome Matte Nail Polish - Aruba'
 'Wanderlust Sand Matte Nail Polish - Raspberry Trifle'
 'Manish Malhotra Gel Finish Nail Lacquer - The Love Edit - Sienna Crush'
 'MyGlamm LIT Nail Enamel - Boi Bye'
 'POPxo Makeup - Dreamin’ Mini Nail Kit']
sku
sgl8906095880447
rsku
Manish Malhotra Nail Lacquer - Teal Tale
sku
sgl8906095880447
rsku
MyGlamm LIT Matte Nail Enamel - Salty
sku
sgl8906095880447
rsku
Manish Malhotra Gel Finish Nail Lacquer - Mint Mistique
sku
sgl8906095880447
rsku
Wanderlust Chrome Matte Nail Polish - Aruba
sku
sgl8906095880447
rsku
Wanderlust Sand Matte Nail Polish - R

In [60]:
df1.iterrows()

<generator object DataFrame.iterrows at 0x000002B99F056AC0>

In [61]:
similar_products

{'8076149815': [],
 '9790153431': [],
 '80761498143': [],
 'STPL1707001': [],
 '8906095887965': [],
 'STPL-1844-001': [],
 'STPL-1845-001': [],
 'STPL-1846-001': [],
 'STPL-1849-001': [],
 'Test2022': [],
 'reward04': [],
 'reward05': [],
 'reward06': [],
 'reward07': [],
 'reward09': [],
 'reward10': [],
 'reward11': [],
 '979015343': [],
 'rewards08': [],
 '80761498324': [],
 'STPL1698001': [],
 'STPL1700001': [],
 'STPL1701001': [],
 'STPL1702001': [],
 'STPL1703001': [],
 'STPL1704001': [],
 '876688822200': [],
 '8906095880485': [],
 '8906095880645': [],
 '8906095880829': [],
 '8906095880836': [],
 '8906095880843': [],
 '8906095880973': [],
 '8906095881109': [],
 '8906095881116': [],
 '8906095881123': [],
 '8906095881505': [],
 '8906095882250': [],
 '8906095882267': [],
 '8906095882274': [],
 '8906095882281': [],
 '8906095882298': [],
 '8906095882304': [],
 '8906095882311': [],
 '8906095882328': [],
 '8906095882335': [],
 '8906095882342': [],
 '8906095882359': [],
 '8906095882366':

In [62]:
empty_rsku = []
for sku, value in similar_products.items(): 
    if value != []:
        empty_rsku.append(sku)

In [63]:
empty_rsku

[]

In [64]:
dump_requests = []

for key in similar_products:
    
    if len(similar_products[key]):
        curl_req_body = {
            "identifier": key,
            "key": "similarProducts",
            "value": {
                "products": similar_products[key]
            },
            "vendorCode":"mgp"
        }
        dump_requests.append(curl_req_body)
        # print(curl_req_body)

f = open("demofile2.txt", "a")
f.write(json.dumps(dump_requests))
f.close()